# DataKind DC - Red Cross Phase 2
## Smoke Alarm Model 1b: Code reproduction

Maria Brun <br>
April, 2018

In [4]:
import pandas as pd
import numpy as np
import scipy
from sklearn import linear_model

#This is old and causes warnings - updated with sklearn.model_selection
# from sklearn.cross_validation import train_test_split, cross_val_score

from sklearn.cross_validation import train_test_split, cross_val_score
#from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import normalize, Imputer
from sklearn.metrics import mean_squared_error
from math import sqrt

#Get the working directory

import os
os.getcwd()

#Change the working directory and confirm it changed to where you want it to be!

os.chdir('C:\\Users\\mcboe72\\Documents\\GitHub\\red-cross-2\\Model-1B')
os.getcwd()

C:\Users\mcboe72\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


'C:\\Users\\mcboe72\\Documents\\GitHub\\red-cross-2'

In [5]:
tract_data = pd.read_csv('https://raw.githubusercontent.com/maria-brun/red-cross-2/master/2009_2013_alarm_presence_by_tract.csv')
del tract_data['Unnamed: 0']
tract_data.head()

,tractid,all_fire_all_years,alarm_unknown_all_years,alarm_presented_all_years,alarm_not_presented_all_years,ratio_no_alarm_in_all,ratio_no_alarm_in_all_known
0,1001020100,4,0,4,0,0.000000,0.000000
1,1001020200,8,1,3,4,0.500000,0.571429
2,1001020300,7,2,0,5,0.714286,1.000000
3,1001020400,8,4,2,2,0.250000,0.500000
4,1001020500,15,5,7,3,0.200000,0.300000


In [3]:
acs = pd.read_csv('http://enigma-public.s3.amazonaws.com/projects/smoke-alarm-risk/data/acs.csv')

In [18]:
acs.head()

,geoid,tenure_renter_occupied,tenure_owner_occupied,vacancy_for_seasonal_recreational,vacancy_for_rent,vacancy_for_sale_only,vacancy_other_vacant,vacancy_rented_not_occupied,vacancy_for_migrant_workers,vacancy_sold_not_occupied,...,hhmove_moved_in_2000_to_2009,hhmove_moved_in_1970_to_1979,hhmove_moved_in_1980_to_1989,hhmove_moved_in_2010_or_later,qfs1_yes,qfs1_no,hdsb_no,hdsb_yes,geoid_parsed,sum_level
0,07000US021986431016360,0.202703,0.797297,0.712871,0.148515,0.000000,0.138614,0.0,0.0,0.000000,...,0.283784,0.000000,0.081081,0.202703,0.108108,0.891892,0.000000,0.033613,021986431016360,07000
1,15000US020900019004,0.142431,0.857569,0.557778,0.126667,0.000000,0.315556,0.0,0.0,0.000000,...,0.603695,0.049464,0.078069,0.155542,NaN,NaN,NaN,NaN,020900019004,15000
2,15000US021220005002,0.206806,0.793194,0.912621,0.000000,0.048544,0.038835,0.0,0.0,0.000000,...,0.500000,0.078534,0.107330,0.117801,NaN,NaN,NaN,NaN,021220005002,15000
3,26900US63150231050,0.371134,0.628866,0.027027,0.081081,0.000000,0.891892,0.0,0.0,0.000000,...,0.505155,0.005155,0.103093,0.231959,0.216495,0.783505,0.015385,0.010256,63150231050,26900
4,26900US64800222910,0.286408,0.713592,0.000000,0.038462,0.000000,0.807692,0.0,0.0,0.153846,...,0.330097,0.179612,0.165049,0.092233,0.359223,0.640777,0.002370,0.008294,64800222910,26900


The geoid is a code that indicates the summary level, the state, county, etc <br>
For reference, see the documentation at this website: https://www.census.gov/geo/reference/geoidentifiers.html <br> <br>
The number of digits determins the level: 2 for state, 5 for state and county, 10 for state and county and county subdivision, etc <br> <br>
This code splits the column geoid into tract ids - parsed ID and sum level <br>
As an example, I output the full geoid and the parsed ID and sum level for the 100th observation in a list

In [86]:

# Get the tract id from the geoid
x = acs['geoid'][100]
parsed_id = x.split('US')[1]
sum_level = x.split('US')[0]


In [87]:
acs['geoid_parsed'] = [geo[1] for geo in acs['geoid'].str.split('US')]
acs['sum_level'] = [geo[0] for geo in acs['geoid'].str.split('US')]

examples_geoid = (acs['geoid'][100], acs['geoid_parsed'][100], acs['sum_level'][100])
examples_geoid

('15000US060374607001', '060374607001', '15000')

In [88]:
#Creates new variables state, county, etc, by pulling out the part of the geoid that corresponds to each 
#Commented out of code

#acs['state'] = acs['geoid_parsed'].str[0:2]
#acs['cnty'] = acs['geoid_parsed'].str[2:5]
#acs['raw_tract'] = acs['geoid_parsed'].str[5:]

In [89]:
#Create a new data set with just the census tract level summary data

acsCT = acs.query("sum_level == '14000'")
acsCT['geoid'].tail()
acsCT['geoid_parsed'].tail()

578840    53033027300
578841    53057952100
578842    53063012300
578866    55007960200
578867    55079017000
Name: geoid_parsed, dtype: object

Thsi code imputes the mean for missing values <br>
First, keep only the features (not the target variable) and create a dataframe with just the target variable. Then, use the imputer to replace missing values with the mean. Then merge the target variable back to the transformed features dataframe.

In [90]:
acsCT = acsCT.rename(columns = {'geoid_parsed':'tractid'})
#This code errors on Windows
#acsCT['tractid'] = acsCT['tractid'].astype(int)

#Instead, use this
acsCT['tractid'] = acsCT['tractid'].astype('int64')


In [91]:

#Create target variable only dataframe
acs_tractid = acsCT['tractid']

#Create features variables only dataframe
acsCT = acsCT.drop(['sum_level', 'geoid'], axis = 1)


In [114]:
acs_features = acsCT.drop(['tractid'], axis = 1)
col_names = acs_features.columns

imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0).fit(acs_features)

acs_features = imp.transform(acs_features)

acsCT = pd.DataFrame(acs_features)
acsCT.columns = col_names
acsCT = pd.concat([acsCT.reset_index(drop=True), acs_tractid.reset_index(drop=True)], axis = 1)

#Added to remove duplicate column 'tractID'
#acsCT.drop(acsCT.columns[len(acsCT.columns)-1], axis=1, inplace=True)
acsCT = acsCT_imp.loc[:,~acsCT.columns.duplicated()]

In [116]:
## Added to verify dimensions

print(acs_features.shape)
len(col_names)
acsCT.head()

(74001, 249)


,tenure_renter_occupied,tenure_owner_occupied,vacancy_for_seasonal_recreational,vacancy_for_rent,vacancy_for_sale_only,vacancy_other_vacant,vacancy_rented_not_occupied,vacancy_for_migrant_workers,vacancy_sold_not_occupied,hhsex_female,...,hhmove_moved_in_1969_or_earlier,hhmove_moved_in_2000_to_2009,hhmove_moved_in_1970_to_1979,hhmove_moved_in_1980_to_1989,hhmove_moved_in_2010_or_later,qfs1_yes,qfs1_no,hdsb_no,hdsb_yes,tractid
0,0.356879,0.643121,0.219365,0.124054,0.128593,0.390318,0.027231,0.0,0.110439,0.500077,...,0.039425,0.444353,0.057084,0.083368,0.205749,0.198357,0.801643,0.019764,0.034319,5015950500
1,0.273465,0.726535,0.259740,0.415584,0.000000,0.324675,0.000000,0.0,0.000000,0.524849,...,0.021437,0.421205,0.084589,0.059096,0.201043,0.092700,0.907300,0.014880,0.016411,5033020302
2,0.619534,0.380466,0.000000,0.469388,0.178571,0.112245,0.239796,0.0,0.000000,0.552688,...,0.023324,0.574344,0.019679,0.032070,0.255102,0.203353,0.796647,0.018452,0.023134,5045030702
3,0.503280,0.496720,0.000000,0.222222,0.057143,0.644444,0.076190,0.0,0.000000,0.466744,...,0.038163,0.586166,0.038163,0.045319,0.162791,0.191413,0.808587,0.021086,0.032425,5057480500
4,0.089406,0.910594,0.212245,0.159184,0.289796,0.220408,0.000000,0.0,0.118367,0.508475,...,0.036176,0.459432,0.055297,0.125581,0.121447,0.067183,0.932817,0.008466,0.016740,5085020103


Merge alarm presence by tract data with the census tract level ACS data

In [95]:
tract_data_merged = tract_data.merge(acsCT, how='left', on='tractid')
tract_data_merged.head()
len(tract_data_merged)

68075

Filter for all fire years greater than 25

Q!

In [96]:
tract_data_merged.isnull().sum()
tract_data_merged_filtered = tract_data_merged.query(" all_fire_all_years > 25 ")
tract_data_merged_filtered.isnull().sum()

tractid                                0
all_fire_all_years                     0
alarm_unknown_all_years                0
alarm_presented_all_years              0
alarm_not_presented_all_years          0
ratio_no_alarm_in_all                  0
ratio_no_alarm_in_all_known          167
tenure_renter_occupied                 5
tenure_owner_occupied                  5
vacancy_for_seasonal_recreational      5
vacancy_for_rent                       5
vacancy_for_sale_only                  5
vacancy_other_vacant                   5
vacancy_rented_not_occupied            5
vacancy_for_migrant_workers            5
vacancy_sold_not_occupied              5
hhsex_female                           5
hhsex_male                             5
built_1980_to_1989                     5
built_1960_to_1969                     5
built_2010_to_later                    5
built_1990_to_1999                     5
built_1950_to_1959                     5
built_1939_or_earlier                  5
built_1940_to_19

In [97]:
tract_data_merged.head()

,tractid,all_fire_all_years,alarm_unknown_all_years,alarm_presented_all_years,alarm_not_presented_all_years,ratio_no_alarm_in_all,ratio_no_alarm_in_all_known,tenure_renter_occupied,tenure_owner_occupied,vacancy_for_seasonal_recreational,...,hhmove_moved_in_1990_to_1999,hhmove_moved_in_1969_or_earlier,hhmove_moved_in_2000_to_2009,hhmove_moved_in_1970_to_1979,hhmove_moved_in_1980_to_1989,hhmove_moved_in_2010_or_later,qfs1_yes,qfs1_no,hdsb_no,hdsb_yes
0,1001020100,4,0,4,0,0.000000,0.000000,0.176948,0.823052,0.0,...,0.159091,0.102273,0.540584,0.053571,0.068182,0.076299,0.141234,0.858766,0.012943,0.019133
1,1001020200,8,1,3,4,0.500000,0.571429,0.363745,0.636255,0.0,...,0.093637,0.063625,0.478992,0.090036,0.126050,0.147659,0.193277,0.806723,0.032741,0.009822
2,1001020300,7,2,0,5,0.714286,1.000000,0.183868,0.816132,0.0,...,0.199480,0.069384,0.454467,0.082394,0.048569,0.145707,0.098873,0.901127,0.027170,0.035785
3,1001020400,8,4,2,2,0.250000,0.500000,0.174286,0.825714,0.0,...,0.190857,0.126857,0.403429,0.112571,0.073714,0.092571,0.038857,0.961143,0.063044,0.053187
4,1001020500,15,5,7,3,0.200000,0.300000,0.407783,0.592217,0.0,...,0.145033,0.007252,0.598985,0.018854,0.010394,0.219483,0.083152,0.916848,0.025199,0.018588


In [98]:
#from sklearn.utils import resample

#high_fires_resampled = resample(tract_data_merged_filtered.query( "all_fire_all_years >= 50" ), n_samples = 1000, random_state = 12)
#tract_data_merged_filtered = tract_data_merged_filtered.append(high_fires_resampled)

#tract_data_merged_filtered.head()

Model Preparation:
Split the dataframe into features versus target variable dataframes

In [99]:

# These are the explanatory variabels

data_features = tract_data_merged_filtered.drop('ratio_no_alarm_in_all', axis = 1)

data_features = data_features.drop(['tractid', 'alarm_unknown_all_years',
                                    'alarm_presented_all_years', 'alarm_not_presented_all_years',
                                    'ratio_no_alarm_in_all_known'], axis = 1)

#This is the target variable

data_target = tract_data_merged_filtered['ratio_no_alarm_in_all']

data_features.isnull().sum()




all_fire_all_years                   0
tenure_renter_occupied               5
tenure_owner_occupied                5
vacancy_for_seasonal_recreational    5
vacancy_for_rent                     5
vacancy_for_sale_only                5
vacancy_other_vacant                 5
vacancy_rented_not_occupied          5
vacancy_for_migrant_workers          5
vacancy_sold_not_occupied            5
hhsex_female                         5
hhsex_male                           5
built_1980_to_1989                   5
built_1960_to_1969                   5
built_2010_to_later                  5
built_1990_to_1999                   5
built_1950_to_1959                   5
built_1939_or_earlier                5
built_1940_to_1949                   5
built_1970_to_1979                   5
built_2000_to_2009                   5
nunits_20_or_more                    5
nunits_3_or_4                        5
nunits_2                             5
nunits_10_to_19                      5
nunits_5_to_9            

In [100]:
## Added to verify dimensions

print(data_features.shape)

(11757, 250)


Handle missing values in the features dataset through imputation based on the mean value

In [101]:

# Handle missing observations by simple mean imputation

col_names = data_features.columns
imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0).fit(data_features)
data_features = imp.transform(data_features)

data_features = pd.DataFrame(data_features)
data_features.columns = col_names

data_features.isnull().sum()

all_fire_all_years                   0
tenure_renter_occupied               0
tenure_owner_occupied                0
vacancy_for_seasonal_recreational    0
vacancy_for_rent                     0
vacancy_for_sale_only                0
vacancy_other_vacant                 0
vacancy_rented_not_occupied          0
vacancy_for_migrant_workers          0
vacancy_sold_not_occupied            0
hhsex_female                         0
hhsex_male                           0
built_1980_to_1989                   0
built_1960_to_1969                   0
built_2010_to_later                  0
built_1990_to_1999                   0
built_1950_to_1959                   0
built_1939_or_earlier                0
built_1940_to_1949                   0
built_1970_to_1979                   0
built_2000_to_2009                   0
nunits_20_or_more                    0
nunits_3_or_4                        0
nunits_2                             0
nunits_10_to_19                      0
nunits_5_to_9            

In [102]:
## Added to verify dimensions

print(data_features_imp.shape)
tract_data_merged_filtered_imp.head()


(11757, 250)


,all_fire_all_years,tenure_renter_occupied,tenure_owner_occupied,vacancy_for_seasonal_recreational,vacancy_for_rent,vacancy_for_sale_only,vacancy_other_vacant,vacancy_rented_not_occupied,vacancy_for_migrant_workers,vacancy_sold_not_occupied,...,hhmove_moved_in_1990_to_1999,hhmove_moved_in_1969_or_earlier,hhmove_moved_in_2000_to_2009,hhmove_moved_in_1970_to_1979,hhmove_moved_in_1980_to_1989,hhmove_moved_in_2010_or_later,qfs1_yes,qfs1_no,hdsb_no,hdsb_yes
0,32.0,0.325733,0.674267,0.227979,0.393782,0.145078,0.111399,0.051813,0.0,0.069948,...,0.181868,0.056460,0.347448,0.080890,0.064604,0.268730,0.121607,0.878393,0.038328,0.047795
1,28.0,0.285122,0.714878,0.168807,0.346789,0.260550,0.183486,0.000000,0.0,0.040367,...,0.096362,0.000000,0.565922,0.019452,0.027017,0.291246,0.037824,0.962176,0.005680,0.019161
2,34.0,0.415825,0.584175,0.253235,0.353050,0.000000,0.062847,0.166359,0.0,0.164510,...,0.107464,0.015152,0.496352,0.026375,0.063973,0.290685,0.030022,0.969978,0.044566,0.038050
3,41.0,0.305115,0.694885,0.000000,0.238739,0.000000,0.621622,0.000000,0.0,0.139640,...,0.216931,0.053616,0.355556,0.067725,0.079012,0.227160,0.041975,0.958025,0.029610,0.038182
4,38.0,0.143726,0.856274,0.334760,0.155822,0.130993,0.182363,0.119863,0.0,0.076199,...,0.177693,0.008112,0.544740,0.044867,0.105196,0.119392,0.187072,0.812928,0.042127,0.028784


## Model

Train / test / split

In [103]:
tract_data_merged_filtered_imp.columns = col_names2
x_train, x_test, y_train, y_test = train_test_split(data_features,
                                                    data_target,
                                                    test_size=0.1,
                                                    random_state=12)

In [104]:
all_fires = x_train.all_fire_all_years
observation_weights = x_train.all_fire_all_years/x_train.all_fire_all_years.sum()
observation_weights.shape

#Don't use this and it errors anyway
#observation_weights_normalized = normalize(observation_weights, norm = 'l1').ravel()
#observation_weights_normalized.shape


scaled_weights = x_train.all_fire_all_years/np.max(x_train.all_fire_all_years)

x_train = x_train.drop(['all_fire_all_years'], axis = 1)
x_test = x_test.drop(['all_fire_all_years'], axis = 1)



In [105]:
len(tract_data_merged_filtered)

11757

In [106]:
# Output to csv
x_train.to_csv('x_train.csv', header = True, index = False)
x_test.to_csv('x_test.csv', header = True, index = False)
y_train.to_csv('y_train.csv', header = True, index = False)
y_test.to_csv('y_test.csv', header = True, index = False)
all_fires.to_csv('all_fires.csv', header = True, index = False)

Linear model - Stochastic Gradient Descent

Q! Why was this model chosen

In [107]:
#######################################
# MODELING
########################################


clf_linear = linear_model.SGDRegressor(loss = 'squared_loss', penalty = 'none', random_state = 12)
clf_linear.fit(x_train, y_train, sample_weight=scaled_weights)
#clf_linear.fit(x_train, y_train, sample_weight=observation_weights_normalized)
#clf_linear.fit(x_train, y_train)
#clf_linear = linear_model.LogisticRegression()
#clf_linear.fit(x_train, y_train)

y_pred = clf_linear.predict(x_test)

# RMSE of simple linear model
rmse = sqrt(mean_squared_error(y_test, y_pred))
print (rmse)


0.15035632560579218


C:\Users\mcboe72\Anaconda\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [108]:

#### Use model to predict for every census tract in the geocoded data (68075) ####

# Clean up original full dataset
tract_data_merged_simple = tract_data_merged.drop([
    'tractid', 'alarm_unknown_all_years','alarm_presented_all_years', 
    'alarm_not_presented_all_years','ratio_no_alarm_in_all_known', 
    'ratio_no_alarm_in_all', 'all_fire_all_years'], axis = 1)                          

tract_data_merged_simple.isnull().sum()

# Handle missing observations by simple mean imputation
col_names = tract_data_merged_simple.columns
imp = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0).fit(tract_data_merged_simple)
tract_data_merged_simple = imp.transform(tract_data_merged_simple)

tract_data_merged_simple = pd.DataFrame(tract_data_merged_simple)
tract_data_merged_simple.columns = col_names                                    
                  
tract_data_merged_simple.isnull().sum()
                  
full_preds = clf_linear.predict(tract_data_merged_simple)

### Bound probabilities
#full_preds[full_preds < 0] = 0
#full_preds[full_preds > 1] = 1

# RMSE of simple linear model on the full dataset
#print sqrt(mean_squared_error(tract_data_merged_simple_clean.ratio_no_alarm_in_all, full_preds))
print (sqrt(mean_squared_error(tract_data_merged.ratio_no_alarm_in_all, full_preds)))



0.23830587738991435


In [110]:
#tract_data['weighted_linear_pred'] = full_preds
tract_data['weighted_linear_pred'] = full_preds
#tract_data_merged['weighted_linear_pred'] = full_preds

tract_data = tract_data.sort_values(['all_fire_all_years'], ascending = False)


tract_data['weighted_linear_pred'].describe()
print (tract_data['weighted_linear_pred'].tail())

# Output to csv
tract_data.to_csv('tract_data_weighted_linear_preds_upsampled.csv', header = True, index = False)


43320    0.047686
52906    0.060209
2372     0.158454
2371     0.026863
15018    0.166931
Name: weighted_linear_pred, dtype: float64


In [117]:
#### Use model to predict for every census tract ####

full_preds_acsCT = clf_linear.predict(acs_features)

### Bound probabilities
#full_preds[full_preds < 0] = 0
#full_preds[full_preds > 1] = 1


# Can't compute RMSE because we have no target for some of the tracts.

tract_preds_74k = pd.concat([acs_tractid.reset_index(drop=True), pd.Series(full_preds_acsCT)], axis = 1)

tract_preds_74k.columns = ['tractid', 'weighted_linear_pred']

tract_preds_74k.weighted_linear_pred.describe()


# Output to csv
tract_preds_74k.to_csv('tracts_74k_weighted_linear_preds_upsampled.csv', header = True, index = False)



